# imports

In [17]:
import pandas as pd
from scipy.stats import truncnorm
import numpy as np 
import matplotlib.pyplot as plt 
from scipy.special import expit as activation_function

# reading the dataframe and making it into a np.array to be able to manipulate easier

In [18]:
df = pd.read_csv("archive/fashion-mnist_train.csv")
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [110]:
df = np.array(df)
m, n = df.shape
np.random.shuffle(df)

data_train = df[1000:m].T
Y_train = data_train[0] 
#normalising the train data so that I can get the values all from 0-1 
X_train = (data_train[1:n]/255).T
#print(sum(X_train))

In [111]:
print(data_train[0])

[9 1 6 ... 6 1 5]


In [112]:
X_train.shape

(59000, 784)

# creating the neural network class

In [119]:
#defining my ReLU
def ReLU(x):
    return np.maximum(0,x)

#defining the derivative of ReLU
def d_ReLU(x):
    return x > 0

#defining the sigmoid activation function
def sigmoid(x):
    return 1 / (1 + np.e ** -x)

#defining the derivative of sigmoid activation function
def d_sigmoid(x):
    return x * (1.0 - x)

#defining the softmax
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis = 0)

def one_hotY(Y):
    one_hot_Y = np.zeros ( (Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

class NeuralNetwork:
    #making the constructor for the NeuralNetwork class
    def __init__(self, no_of_in_nodes, no_of_out_nodes, no_of_hidden_nodes, learning_rate):
        self.no_of_in_nodes = no_of_in_nodes
        self.no_of_out_nodes = no_of_out_nodes
        self.no_of_hidden_nodes = no_of_hidden_nodes
        self.learning_rate = learning_rate 
        self.create_weight_matrices()
    
    def create_weight_matrices(self):
        #working out the weight
        self.w1 = np.random.rand(784, 10) - 0.5 
        #working out the bias for each weight
        self.b1 = np.random.rand(1, 10) - 0.5
        self.w2 = np.random.rand(10, 10) - 0.5 
        self.b2 = np.random.rand(1, 10) - 0.5
    
    def train(self, input_vector, target_vector,epochs=20):
        print ("shape of w1 {}".format(self.w1.shape))
        print ("shape of w2 {}".format(self.w2.shape))
        
        print ("shape of X {}".format(input_vector.shape))
        print ("shape of y {}".format(target_vector.shape))
        
        
        
        
        
        for i in range(epochs):
            #forward pass
            z1 = self.w1.dot(input_vector) + self.b1
            a1 = ReLU(z1)
            z2 = self.w2.dot(a1) + self.b2
            a2 = softmax(z2)
            
            #a2_delta = (target_vector - a2) * (a2*(1-a2))
            #print(a2_delta.shape)
            #print('W2 {}'.format(self.w2.T.shape))
            #a1_delta = a2_delta.dot(self.w2.T) * (a1 * (1-a1))
            
            one_hot_Y = one_hotY(Y_train)
            
            #backward prop
            dz2 = a2 - one_hot_Y
            dw2 = 1/m * dz2.dot(a1.T)
            db2 = 1/m * np.sum(dz2)
            dz1 = self.w2.T.dot(dz2) * derivative_ReLU(z1)
            dw1 = 1/m * dz1.dot(input_vector.T)
            db1 = 1/m * np.sum(dz1)
            
            #updating the weight
            self.w1 = self.w1 - self.learning_rate * dw1
            self.w2 = self.w2 - self.learning_rate * dw2

        print ("shape of layer1 activation {}".format(a1.shape))
        print ("shape of output layer  {}".format(a2.shape))
        
        

        #print ("shape of output layer error (a2_delta) {}".format(a2_delta.shape))
        #print ("shape of a1 derivative  {}".format(a1_delta.shape))
    def run(self, input_vector):
        output_vector = np.dot(self.w1, input_vector)
        output_vector = sigmoid(output_vector)
        
        output_vector = np.dot(self.w2, output_vector)
        output_vector = sigmoid(output_vector)
        
        return output_vector.T
    
    #def evaluate(self, data, labels):
     #   corrects, wrongs = 0, 0
      #  for i in range(len(data)):
       #     res = self.run(data[i])
        #    res_max = res.argmax()
         #   if res_max == labels[i]:
          #      corrects += 1
           # else:
            #    wrongs += 1
       # return corrects, wrongs
   

In [120]:
model = NeuralNetwork(X_train,Y_train, 10 ,0.1)
model.train(X_train, Y_train)
y_hat = model.run(X_train)
#print(y_hat)

shape of w1 (784, 10)
shape of w2 (10, 10)
shape of X (59000, 784)
shape of y (59000,)


ValueError: shapes (784,10) and (59000,784) not aligned: 10 (dim 1) != 59000 (dim 0)